In [ ]:
## This code is for running the following linear regression models: 
#Loneliness ~ functional connectivity (roi-to-roi pair) in run 1 + covariates (age + sex + condition)​
#Loneliness ~ functional connectivity (roi-to-roi pair) in run 2 + covariates (age + sex + condition)​
#Social connectedness ~ functional connectivity (roi-to-roi pair) in run 1 + covariates (age + sex + condition) ​
#Social connectedness ~ functional connectivity (roi-to-roi pair) in run 2 + covariates (age + sex + condition) 

In [ ]:
##required imports

In [ ]:
#Path on your device with all subjects' individual connectivity matrices  
connectivity_dir_all = '/path/to/all/functional_connectivity_matrices' ##directory with all participants' functional connectivity matrices 

#grab all of the .csv files (the connectivity matrices) in that directory 
connectivity_files_all = sorted(glob.glob(os.path.join(connectivity_dir_all, '*.csv')))

In [ ]:
# loading all of the functional connectivity matrices (69x69 ROI-to-ROI, 2 per subject (run 1 and run 2)) into a single 3D NumPy array for analysis 
all_matrices = [pd.read_csv(f).values for f in connectivity_files_all]
all_connectivity_group = np.stack(all_matrices)  #takes the matrices and stacks them into a single 3D NumPy array of shape 

In [ ]:
#Vectorize connectivity matrices 
#converting each 69x69 matrix into a 1D vector of unique connections, obtaining a feature vector for each subject

import numpy as np 

def vectorize_connectivity(matrices):
    n_subjects, n_rois, _ = matrices.shape
    triu_idx = np.triu_indices(n_rois, k=1)  # indices of upper triangle without diagonal
    n_edges = len(triu_idx[0])

    X = np.zeros((n_subjects, n_edges))
    for i in range(n_subjects):
        X[i] = matrices[i][triu_idx]
    return X, triu_idx

X, triu_idx = vectorize_connectivity(all_connectivity_group)
print('Shape of feature matrix X:', X.shape)  # (N_subjects, 2346)

In [ ]:
##splitting the vectors by run (run 1 is self, run 2 is other)
run1_indices = np.arange(0, X.shape[0], 2)  # 0, 2, 4, ...
run2_indices = np.arange(1, X.shape[0], 2)  # 1, 3, 5, ...

X_run1 = X[run1_indices]
X_run2 = X[run2_indices]

In [ ]:
##loading in the behavioural data (i.e., the spreadsheet)
behaviour_df = pd.read_csv('/insert/your/path/to/Dataset.csv') #"Dataset.csv" is the spreadsheet containing the behavioural data (ex. loneliness, connectedness sum scores)
behaviour_df = behaviour_df.iloc[0:54] #keeping the first 55 rows that have the behavioural data for each participant

In [ ]:
##preparing outcomes of interest and covariates 
#behaviour vector
#just using loneliness_sum_T1 as a test for dependent variable
loneliness_sum_T1 = behaviour_df['loneliness_sum_T1'].values

#covariates 
covariates = pd.get_dummies(behaviour_df[['age', 'Sex', 'condition']], drop_first=True)

In [ ]:
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests

#Below, I am defining the linear regression function that I will be using 
#X_run (x, independent variable) and loneliness_sum_T1 (y, dependent variable) are just like placeholders;
#you can specify any x or y (ex. x = run 1 or run 2, and/or y = loneliness or social connectedness) later on
def run_featurewise_regression_with_covariates(X_run, loneliness_sum_T1, covariates):
    betas = []
    tstats = []
    pvals = []
    #convert y (loneliness_sum_T1) to numeric numpy array (float)
    if isinstance(loneliness_sum_T1, (pd.Series, pd.DataFrame)):
        loneliness_sum_T1_numeric = pd.to_numeric(loneliness_sum_T1, errors='coerce').values.astype(float)
    else:
        loneliness_sum_T1_numeric = np.array(loneliness_sum_T1, dtype=float)
    
    #convert covariates to numeric dummy variables (if needed) and then numpy float array 
    #ensure covariates is a NumPy array 
    if isinstance(covariates, pd.DataFrame):
        covariates_numeric = pd.get_dummies(covariates, drop_first=True)
        covariates_array = covariates_numeric.values.astype(float)
    else:
        covariates_array = np.array(covariates, dtype=float)

    # Ensure connectivity matrix is float
    X_run = X_run.astype(float)
    
    for i in range(X_run.shape[1]):
        #stack current connectivity feature with covariates
        Xi = np.column_stack((X_run[:, i], covariates_array))
        Xi = sm.add_constant(Xi) #adds intercept term

        #fit regression model: Loneliness ~ connectivity feature + covariates
        #ordinary least squares regression model 
        model = sm.OLS(loneliness_sum_T1, Xi).fit()

        #extract beta and p-value for connectivity feature (index 1 after constant)
        betas.append(model.params[1])    # coefficient for connectivity feature
        tstats.append(model.tvalues[1])
        pvals.append(model.pvalues[1])   # p-value for connectivity feature

    return np.array(betas), np.array(tstats), np.array(pvals)

In [ ]:
##here, we are calling upon all the ROIs (cortical and subcortical) in the harvard_oxford atlas,
##which was originally used to create our functional connectivity matrices
cort = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
sub = datasets.fetch_atlas_harvard_oxford('sub-maxprob-thr25-2mm')

cort_labels = cort.labels[1:]
sub_labels = sub.labels[1:]
atlas_labels = cort_labels + sub_labels

print(f"Total ROI labels: {len(atlas_labels)}")

In [ ]:
##In total, each connectivity matrix has 69 ROIs (69 x 69)
##Here, we are extracting a list of the roi pairs (ex. (ROI 1, ROI2))
n_rois = len(atlas_labels)
roi_pairs = []

# Get upper triangle indices (exclude diagonal)
triu_indices = np.triu_indices(n_rois, k=1)

for idx in range(len(triu_indices[0])):
    i = triu_indices[0][idx]
    j = triu_indices[1][idx]
    roi_pairs.append((atlas_labels[i], atlas_labels[j]))

In [ ]:
#Our ROI paits of interest are ones that contain either the 'Insula' or 'Cingulate' as one of the pairs
#(based on prior research, functional connectivity between the insula and the cingulate cortex with other brain regions is implicated in self and empathic pain)
#so we want to select specifically for roi pairs that contain either 'Insula' or 'Cingulate'
selected_pairs = [
    pair for pair in roi_pairs
    if ('Insula' in pair[0] or 'Cingulate' in pair[0] or ('Insula' in pair[1] or 'Cingulate' in pair[1]))
]

print(f"Number of ROI pairs with 'Insula' or 'Cingulate': {len(selected_pairs)}")

# Indices of selected_pairs in roi_pairs as well:
selected_indices = [
    idx for idx, pair in enumerate(roi_pairs)
    if ('Insula' in pair[0] or 'Cingulate' in pair[0]) or ('Insula' in pair[1] or 'Cingulate' in pair[1])
]

for idx in selected_indices:
    print(f"Index: {idx}, ROI Pair: {roi_pairs[idx]}")

In [ ]:
#Subset run-specific data to only include selected ROI pairs 
#Now, we are subsetting our dataframe, so that only the functional connectivity measures of those selected ROI pairs 
#are included in our linear regression models 
X_run1_subset = X_run1[:, selected_indices]
X_run2_subset = X_run2[:, selected_indices]

In [ ]:
##Now, calling upon our behaviour scores of interest (Loneliness and Connectedness) from our behaviour dataframe 
Loneliness_sumT2 = behaviour_df['Loneliness_sumT2'].values
Connectedness_sum_T2 = behaviour_df['Connectedness_sum_T2'].values

In [ ]:
#Running linear regressions with loneliness as dependent variable: 
# Loneliness ~ functional connectivity (roi-to-roi pair) in run 1 + covariates (age + sex + condition)
betas_run1_loneliness_T2, tstats_run1_loneliness_T2, pvals_run1_loneliness_T2 = run_featurewise_regression_with_covariates(
    X_run1_subset, Loneliness_sumT2, covariates
)

#Loneliness ~ functional connectivity (roi-to-roi pair) in run 2 + covariates (age + sex + condition)
betas_run2_loneliness_T2, tstats_run2_loneliness_T2, pvals_run2_loneliness_T2 = run_featurewise_regression_with_covariates(
    X_run2_subset, Loneliness_sumT2, covariates
)

In [ ]:
#Running linear regressions with social connectedness as the dependent variable:
#Social connectedness ~ functional connectivity (roi-to-roi pair) in run 1 + covariates (age + sex + condition) 
betas_run1_connectedness_T2, tstats_run1_connectedness_T2, pvals_run1_connectedness_T2 = run_featurewise_regression_with_covariates(
    X_run1_subset, Connectedness_sum_T2, covariates
)

#Social connectedness ~ functional connectivity (roi-to-roi pair) in run 2 + covariates (age + sex + condition) 
betas_run2_connectedness_T2, tstats_run2_connectedness_T2, pvals_run2_connectedness_T2 = run_featurewise_regression_with_covariates(
    X_run2_subset, Connectedness_sum_T2, covariates
)

In [ ]:
#Storing the results for linear regressions (loneliness as dependent variable) in a dataframe
results_run1_loneliness_T2 = pd.DataFrame({
    'roi_pair': roi_pair_names,
    'beta': betas_run1_loneliness_T2,
    't_stat': tstats_run1_loneliness_T2,
    'p_value': pvals_run1_loneliness_T2
})

results_run2_loneliness_T2 = pd.DataFrame({
    'roi_pair': roi_pair_names,
    'beta': betas_run2_loneliness_T2,
    't_stat': tstats_run2_loneliness_T2,
    'p_value': pvals_run2_loneliness_T2
})

In [ ]:
#Storing results for linear regressions (social connectedness as dependent variable) in a dataframe
results_run1_connectedness_T2 = pd.DataFrame({
    'roi_pair': roi_pair_names,
    'beta': betas_run1_connectedness_T2,
    't_stat': tstats_run1_connectedness_T2,
    'p_value': pvals_run1_connectedness_T2
})

results_run2_connectedness_T2 = pd.DataFrame({
    'roi_pair': roi_pair_names,
    'beta': betas_run2_connectedness_T2,
    't_stat': tstats_run2_connectedness_T2,
    'p_value': pvals_run2_connectedness_T2
})

In [ ]:
##Saving Results  
results_run1_loneliness_T2.to_csv("/path/to/where/you/want/to/save/your/results/results_run1_loneliness_T2.csv")
results_run2_loneliness_T2.to_csv("/path/to/where/you/want/to/save/your/results/results_run2_loneliness_T2.csv")
results_run1_connectedness_T2.to_csv("/path/to/where/you/want/to/save/your/results/results_run1_connectedness_T2.csv")
results_run2_connectedness_T2.to_csv("/path/to/where/you/want/to/save/your/results/results_run2_connectedness_T2.csv")